# Pipeline SDK

In [1]:
import kfp

@kfp.dsl.pipeline(
    name='hello_my_first_pipeline', 
    description='mxnet mnist sample pipeline'
)

def hello_pipeline():
    train_task = hello_component()

@kfp.dsl.component
def hello_component():
    return kfp.dsl.ContainerOp(
        name='my-ContainerOp',
        # Feb 26, 2023 version
        image='kubeflowkatib/mxnet-mnist:v1beta1-2e27185',
        # required for emissary executor.
        command=[
            'python',
            '/opt/mxnet-mnist/mnist.py'
        ]
    )

# Pipeline 컴파일

In [2]:
kfp.compiler.Compiler().compile(hello_pipeline, 'my_pipeline.zip')

/opt/conda/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1261: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(


In [3]:
! ls -l my_pipeline.zip

-rw-r--r-- 1 jovyan users 597 Mar 11 09:20 my_pipeline.zip


In [4]:
! unzip -l my_pipeline.zip

Archive:  my_pipeline.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
     1102  1980-01-01 00:00   pipeline.yaml
---------                     -------
     1102                     1 file


In [5]:
! unzip -p my_pipeline.zip

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: hello-my-first-pipeline-
  annotations: {pipelines.kubeflow.org/kfp_sdk_version: 1.8.19, pipelines.kubeflow.org/pipeline_compilation_time: '2023-03-11T09:20:34.355858',
    pipelines.kubeflow.org/pipeline_spec: '{"description": "mxnet mnist sample pipeline",
      "name": "hello_my_first_pipeline"}'}
  labels: {pipelines.kubeflow.org/kfp_sdk_version: 1.8.19}
spec:
  entrypoint: hello-my-first-pipeline
  templates:
  - name: hello-my-first-pipeline
    dag:
      tasks:
      - {name: my-containerop, template: my-containerop}
  - name: my-containerop
    container:
      command: [python, /opt/mxnet-mnist/mnist.py]
      image: kubeflowkatib/mxnet-mnist:v1beta1-2e27185
    metadata:
      labels:
        pipelines.kubeflow.org/kfp_sdk_version: 1.8.19
        pipelines.kubeflow.org/pipeline-sdk-type: kfp
        pipelines.kubeflow.org/enable_caching: "true"
      annotations: {pipelines.kubeflow.org/component_spec:

# Pipeline Experiment

In [6]:
client = kfp.Client()

my_experiment = client.create_experiment(
    name='My First Experiment' # Experiment name
)

print(my_experiment)

{'created_at': datetime.datetime(2023, 3, 11, 7, 35, 31, tzinfo=tzlocal()),
 'description': None,
 'id': '1c176a8b-fca6-45a5-b855-84e241e10531',
 'name': 'My first Experiment',
 'resource_references': [{'key': {'id': 'pjt-lecture', 'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}


# Pipeline 실행

In [7]:
my_run = client.run_pipeline(
    my_experiment.id, 
    'hello_my_pipeline', # Run name
    'my_pipeline.zip'
)